Dieses Notebook berechnet auf Basis eines Datensatzes von 350.000 deutschen Lemmata, die automatisch mit einem Score für die Kategorie "Concreteness" versehen wurden Konkretheitswerte für 1147 deutschsprachige Romane.

the dataset used to compute the scores can be downloaded from this website: http://www.schulteimwalde.de/resources/affective-norms.html

In [1]:
import pandas as pd
from tqdm import tqdm
import re

In [2]:
# read the dataset
dfca = pd.read_csv("data/ratings_lrec16_koeper_ssiw.txt", sep="\t")

dfca_dict = {}
for i in range(len(dfca)):
    key = dfca["Word"].loc[i]
    value = dfca["AbstConc"].loc[i]
    dfca_dict[key] = value


In [3]:
# lemmatisiertes Korpus einlesen

df = pd.read_csv("data/roman_corpus_lemmatisiert.csv", sep=";")

# Listen wiederherstellen
for i in tqdm(range(len(df))):
    lemmas = str(df["all_lemmas"].loc[i])
    lemmas = re.sub(r"'", "", lemmas)
    lemmas = re.sub(r"\[", "", lemmas)
    lemmas = re.sub(r"\]", "", lemmas)
    lemmas = lemmas.split(", ")
    df["all_lemmas"].loc[i] = lemmas
    #print(type(lemmas))

  0%|                                                  | 0/1147 [00:00<?, ?it/s]/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3484/3705137880.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["all_lemmas"].loc[i] = lemmas
100%|███████████████████████████████████████| 1147/1147 [00:18<00:00, 60.61it/s]


In [4]:
# Diese Schleife iteriert über alle Romane des Korpus. Sie überprüft für jedes Lemma eines Romans, 
# ob es im Datensatz enthalten ist. Falls es enthalten ist, wird der dort angegebene Konkretheitswert zu den bisherigen
# Konkretheitswerten des Romans hinzuaddiert. Am Ende werden die KOnkretheitswerte normalisiert, indem sie durch
# die Anzahl der Wörter im jeweiligen Roman geteilt werden.

df["AbstConc"] = 0

for i in tqdm(range(len(df))):
    lemmas = df["all_lemmas"].loc[i]
    for lemma in lemmas:
        if lemma in dfca_dict.keys():
            df["AbstConc"].loc[i] += dfca_dict[lemma]
            
df["anteil_AbstConc"] = 0
for i in range(len(df)):
    df["anteil_AbstConc"].loc[i] = df["AbstConc"].loc[i]/df["all_words_count"].loc[i]

  0%|                                                  | 0/1147 [00:00<?, ?it/s]/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3484/1654838879.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["AbstConc"].loc[i] += dfca_dict[lemma]
100%|███████████████████████████████████████| 1147/1147 [42:31<00:00,  2.22s/it]
/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3484/1654838879.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["anteil_AbstConc"].loc[i] = df["AbstConc"].loc[i]/df["all_words_count"].loc[i]


In [7]:
# speichern
df = df.drop(columns=["all_lemmas"])
df.to_csv("data/roman_corpus_AbstConc.csv", sep=";")